# Amazon Sales Data Analysis

This notebook performs data cleaning, exploratory data analysis (EDA), and visualization for the Amazon Sales Dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Settings
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

## 1. Data Loading and Cleaning

In [ ]:
source_dir = r"d:/all data science project/Sales dataset/sales dataset"
output_dir = r"d:/all data science project/Sales dataset/cleaned_data"
os.makedirs(output_dir, exist_ok=True)

files = glob.glob(os.path.join(source_dir, "*.csv"))

def clean_sales_data(df, filename):
    # Standardize column names
    df.columns = [c.strip() for c in df.columns]
    
    # Date parsing
    date_cols = [c for c in df.columns if 'date' in c.lower()]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Numeric conversion
    numeric_cols = ['Amount', 'Qty', 'PCS', 'RATE', 'GROSS AMT', 'mrp', 'amount']
    for col in df.columns:
        if any(n in col.lower() for n in numeric_cols):
             df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop fully empty entries
    df.dropna(how='all', inplace=True)
    
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    return df

cleaned_dfs = {}

for file_path in files:
    filename = os.path.basename(file_path)
    try:
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except:
            df = pd.read_csv(file_path, encoding='ISO-8859-1')
            
        cleaned_df = clean_sales_data(df, filename)
        
        # Save cleaned file
        output_path = os.path.join(output_dir, f"cleaned_{filename}")
        cleaned_df.to_csv(output_path, index=False)
        cleaned_dfs[filename] = cleaned_df
        print(f"Processed {filename} -> {output_path}")
        
    except Exception as e:
        print(f"Failed to process {filename}: {e}")

## 2. Analysis and Visualization (Amazon Sales Report)

In [ ]:
# Focus on the main dataset
df_sales = cleaned_dfs.get('Amazon-Sale-Report.csv')

if df_sales is not None:
    # Monthly Sales Trend
    df_sales['Month'] = df_sales['Date'].dt.to_period('M')
    monthly_sales = df_sales.groupby('Month')['Amount'].sum()
    
    plt.figure(figsize=(12, 6))
    monthly_sales.plot(kind='line', marker='o')
    plt.title('Monthly Sales Trend')
    plt.ylabel('Amount')
    plt.show()
    
    # Top 10 Products by Quantity
    top_products = df_sales.groupby('SKU')['Qty'].sum().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    top_products.plot(kind='bar')
    plt.title('Top 10 Selling Products (Qty)')
    plt.ylabel('Quantity')
    plt.show()
    
    # Sales by Category
    cat_sales = df_sales.groupby('Category')['Amount'].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 6))
    cat_sales.plot(kind='bar')
    plt.title('Sales by Category')
    plt.ylabel('Amount')
    plt.show()
    
    # Sales by State
    state_sales = df_sales.groupby('ship-state')['Amount'].sum().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    state_sales.plot(kind='barh')
    plt.title('Top 10 States by Sales')
    plt.show()
else:
    print("Amazon-Sale-Report.csv not found or failed to load.")